In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38018")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for NIKE, Inc. North America Distribution Centers (NADC) Undergraduate Internship

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsInternshipsNIKE, Inc. North America Distribution 

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys:
    'role','experience','skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "NIKE, Inc. North America Distribution Centers (NADC) Undergraduate Internship",
  "experience": "0-2 years",
  "skills": [
    "SQL",
    "Excel",
    "Tableau",
    "SAP",
    "Kronos",
    "Alteryx",
    "Snowflake",
    "Lean Manufacturing",
    "labor management",
    "AutoCAD",
    "Analytical skillset",
    "Statistical processes"
  ],
  "description": "As an NADC Undergraduate Intern, you’ll work in a dynamic, growing, and fast-paced distribution center environment. You will utilize your expertise to work on the following types of projects: Operational process improvements, Project deployment such as new technology or capabilities, Improving information and data flows across business and operational partners, or even sustainability initiatives, Continuous Improvement, Labor Management, Material Handling Equipment (MHE) Design."
}
```


In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'NIKE, Inc. North America Distribution Centers (NADC) Undergraduate Internship',
 'experience': '0-2 years',
 'skills': ['SQL',
  'Excel',
  'Tableau',
  'SAP',
  'Kronos',
  'Alteryx',
  'Snowflake',
  'Lean Manufacturing',
  'labor management',
  'AutoCAD',
  'Analytical skillset',
  'Statistical processes'],
 'description': 'As an NADC Undergraduate Intern, you’ll work in a dynamic, growing, and fast-paced distribution center environment. You will utilize your expertise to work on the following types of projects: Operational process improvements, Project deployment such as new technology or capabilities, Improving information and data flows across business and operational partners, or even sustainability initiatives, Continuous Improvement, Labor Management, Material Handling Equipment (MHE) Design.'}

In [15]:
type(json_res)

dict

In [17]:
import pandas as pd

df= pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection= client.get_or_create_collection(name='my_portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                                  ids=[str(uuid.uuid4())])

In [21]:
links = collection.query(query_texts=["Experience in Python","Expertise in React"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [22]:
job = json_res
job['skills']

['SQL',
 'Excel',
 'Tableau',
 'SAP',
 'Kronos',
 'Alteryx',
 'Snowflake',
 'Lean Manufacturing',
 'labor management',
 'AutoCAD',
 'Analytical skillset',
 'Statistical processes']

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Operational Excellence for NIKE, Inc. NADC with AtliQ's Expertise

Dear Hiring Manager,

I came across the NIKE, Inc. North America Distribution Centers (NADC) Undergraduate Internship job description and was impressed by the emphasis on leveraging technology to drive operational process improvements, labor management, and sustainability initiatives. As a Business Development Executive at AtliQ, I'd like to introduce our company's capabilities in empowering businesses like NIKE, Inc. to achieve scalability, process optimization, and heightened efficiency.

AtliQ has a proven track record of delivering tailored solutions that cater to the unique needs of our clients. Our expertise in data analytics, process automation, and software development can help NIKE, Inc. NADC streamline operations, enhance labor management, and improve information flows across business and operational partners.

Our team has hands-on experience with tools like SQL, Excel, Tableau, SAP, and Al